In [1]:
import re
import pandas as pd
import numpy as np

# # from gensim.models import KeyedVectors
# from nltk.corpus import stopwords
# from nltk.stem import SnowballStemmer
# from keras.preprocessing.text import Tokenizer
# from keras.preprocessing.sequence import pad_sequences
# from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation
# from keras.layers.merge import concatenate
# from keras.layers.normalization import BatchNormalization
# from keras.models import Model
# from keras.callbacks import EarlyStopping, ModelCheckpoint

In [2]:
def clean_wordlist(text, remove_stopwords=False, stem_words=False):
    """将文本进行清洗处理

    Args:
        text: 输入文本 str
        remove_stopwords: 删除停用词
        stem_words: 提取词干（去除词后缀）

    Returns: 清洗后的数据 str

    """
    text = text.lower().split()

    if remove_stopwords:
        stops = set(stopwords.words('english'))
        text = [w for w in text if w not in stops]

    # clean the text
    text = ' '.join(text)

    text = re.sub(r"[^A-Za-z0-9^,!./'+-=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", "not ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ", text)
    text = re.sub(r"/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"-", " - ", text)
    text = re.sub(r"=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r" e g", " eg ", text)
    text = re.sub(r" b g", " bg ", text)
    text = re.sub(r" u s", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)

    if stem_words:
        text = text.split()
        stemmer = SnowballStemmer('english')
        stemmed_words = [stemmer.stem(word) for word in text]
        text = ' '.join(stemmed_words)

    return text

In [2]:
# define some constant ########################
#
FILE_PATH = 'd:/work/source/kaggle/quora/'
EMBEDDING_PATH = 'd:/work/source/word_vectors/glove.6B.100d.txt'
MAX_SEQUENCE_LENGTH = 30
MAX_NB_WORDS = 20000  # 处理的最大单词数量
EMBEDDING_DIM = 100  # EMBEDDING_PATH的vocab的维度
VALIDATION_SPLIT = 0.1

nums_lstm = np.random.randint(175, 275)
nums_dense = np.random.randint(100, 150)
rate_drop_lstm = 0.15 + np.random.rand() * 0.25
rate_drop_dense = 0.15 + np.random.rand() * 0.25
STAMP = 'lstm_{}_{}_{:.2f}_{:.2f}'.format(nums_lstm, nums_dense, rate_drop_lstm, rate_drop_dense)

acti = 'relu'
re_weight = True  # whether to re-weight classes to fit the 17.5% share in test set

In [4]:
# index word vectors ###########################
#
print('Indexing word vectors...')
# word2vec = KeyedVectors.load_word2vec_format(EMBEDDING_PATH, binary=True)
embedding_index = dict()  # 这是一个每个单词对应一个100维的向量
f = open(EMBEDDING_PATH, 'r', encoding='utf8')
for line in f:
    values = line.split()
    word = values[0]
    embedding_index[word] = np.asarray(values[1:], dtype='float32')
print('Found {} word vectors of word2vec\n'.format(len(embedding_index)))

Indexing word vectors...


Found 400000 word vectors of word2vec



In [9]:
len(test_text) // 2

2345796

In [8]:
# process text in dateset
#
print('Processing text dateset...\n')
train_df = pd.read_csv(FILE_PATH + 'train.csv')
test_df = pd.read_csv(FILE_PATH + 'test.csv')
# train_df[['question1', 'question2']] = train_df[['question1', 'question2']].\
#     astype(str).applymap(clean_wordlist)
# test_df[['question1', 'question2']] = test_df[['question1', 'question2']].\
#     astype(str).applymap(clean_wordlist)

train_text1 = train_df['question1'].tolist()
train_text2 = train_df['question2'].tolist()
test_text1 = test_df['question1'].tolist()
test_text2 = test_df['question2'].tolist()
# train_text = ' '.join(train_df['question1']) + ' '.join(train_df['question2'])
# test_text = ' '.join(test_df['question1']) + ' '.join(test_df['question2'])
# train_text = pd.concat([train_text1, train_text2])
# test_text = pd.concat([test_text1 + test_text2])
train_text = train_text1 + train_text2
test_text = test_text1 + test_text2
label = train_df['is_duplicate']

# # Tokenizer #################################
# # 把所有的单词对应到索引 -> 一个句子就是一个二维向量
# #
# print('Starting token...')
# tokenizer = Tokenizer(num_words=MAX_NB_WORDS)  # 处理最常见的NB个单词
# tokenizer.fit_on_texts(train_text + test_text)  # 处理后每个单词对应一个索引（一个整数）
# word_index = tokenizer.word_index
# print('Found {} unique tokens\n'.format(len(word_index)))  # train + test全部传入

Processing text dateset...



In [28]:
train_data.shape

(808580, 30)

In [7]:
train_sequences = tokenizer.texts_to_sequences(train_text)
test_sequences = tokenizer.texts_to_sequences(test_text)

print('Starting pad seq...\n')
train_data = pad_sequences(train_sequences, MAX_SEQUENCE_LENGTH)
test_data = pad_sequences(test_sequences, MAX_SEQUENCE_LENGTH)

Starting pad seq...



In [11]:
print(train_data[0])
print(train_sequences[0])

[   0    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    2    3    1 1259   60 1259 2932    8  580    7  762  372    7   34]
[2, 3, 1, 1259, 60, 1259, 2932, 8, 580, 7, 762, 372, 7, 34]


In [12]:
# embedding #################################
#
MAX_NB_WORDS = 200000
print('Preparing embedding matrix...')
nb_words = min(MAX_NB_WORDS, len(word_index))
embedding_matrix = np.zeros((nb_words + 1, EMBEDDING_DIM))

for word, i in word_index.items():
    if word in embedding_index:
        embedding_matrix[i] = embedding_index[word]

# np.sum 沿着轴方向，分别求轴上每一个元素的其他轴上的和
print('Null word embedding: {}\n'.format(np.sum(np.sum(embedding_matrix, axis=1) == 0)))

Preparing embedding matrix...


Null word embedding: 43813



In [16]:
embedding_matrix[1]

array([-0.038194  , -0.24487001,  0.72812003, -0.39961001,  0.083172  ,
        0.043953  , -0.39140999,  0.3344    , -0.57545   ,  0.087459  ,
        0.28786999, -0.06731   ,  0.30906001, -0.26383999, -0.13231   ,
       -0.20757   ,  0.33395001, -0.33848   , -0.31742999, -0.48335999,
        0.1464    , -0.37303999,  0.34577   ,  0.052041  ,  0.44946   ,
       -0.46970999,  0.02628   , -0.54154998, -0.15518001, -0.14106999,
       -0.039722  ,  0.28277001,  0.14393   ,  0.23464   , -0.31020999,
        0.086173  ,  0.20397   ,  0.52623999,  0.17163999, -0.082378  ,
       -0.71787   , -0.41531   ,  0.20334999, -0.12763   ,  0.41367   ,
        0.55186999,  0.57907999, -0.33476999, -0.36559001, -0.54856998,
       -0.062892  ,  0.26583999,  0.30204999,  0.99774998, -0.80480999,
       -3.0243001 ,  0.01254   , -0.36941999,  2.21670008,  0.72201002,
       -0.24978   ,  0.92136002,  0.034514  ,  0.46744999,  1.10790002,
       -0.19358   , -0.074575  ,  0.23353   , -0.052062  , -0.22

In [34]:
# sample data ###############################
#
perm = np.random.permutation(len(train_text1))  # 随机重排序列，也就是进行随机采样
mid = int(len(train_text1) * (1 - VALIDATION_SPLIT))
idx_train = perm[: mid]
idx_valid = perm[mid:]
mid_data = len(train_data) // 2
train_text1 = train_data[: mid_data]
train_text2 = train_data[mid_data:]

train_data1 = np.vstack([train_text1[idx_train], train_text2[idx_train]])
train_data2 = np.vstack([train_text2[idx_train], train_text1[idx_train]])
train_label = np.concatenate([label[idx_train], label[idx_train]])

valid_data1 = np.vstack([train_text1[idx_valid], train_text2[idx_valid]])
valid_data2 = np.vstack([train_text2[idx_valid], train_text1[idx_valid]])
valid_label = np.concatenate([label[idx_valid], label[idx_valid]])

valid_weight = np.ones(len(valid_label))
if re_weight:
    valid_weight *= 0.472001959
    valid_weight[valid_label == 0] = 1.309028344

In [19]:
# define model ###############################
#

# 经过embedding会将二维输入数据变成三维数据
embedding_layer = Embedding(nb_words + 1, EMBEDDING_DIM, weights=[embedding_matrix], input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False)
lstm_layer = LSTM(nums_lstm, dropout=rate_drop_lstm, recurrent_dropout=rate_drop_lstm)

sequence_input_1 = Input(shape=[MAX_SEQUENCE_LENGTH], dtype='int32')
embedding_sequence_1 = embedding_layer(sequence_input_1)
x1 = lstm_layer(embedding_sequence_1)

sequence_input_2 = Input(shape=[MAX_SEQUENCE_LENGTH], dtype='int32')
embedding_sequence_2 = embedding_layer(sequence_input_2)
x2 = lstm_layer(embedding_sequence_2)

merged = concatenate([x1, x2])
merged = Dropout(rate_drop_dense)(merged)
merged = BatchNormalization()(merged)
merged = Dense(nums_dense, activation=acti)(merged)  # 全连接层
merged = Dropout(rate_drop_dense)(merged)
merged = BatchNormalization()(merged)

preds = Dense(1, activation='sigmoid')(merged)

In [20]:
# add class weight #############################
#
if re_weight:
    class_weight = {
        0: 1.309028344,
        1: 0.472001959,
    }
else:
    class_weight = None

In [21]:
# train ########################################
#
model = Model(inputs=[sequence_input_1, sequence_input_2], outputs=preds)
model.compile(optimizer='nadam', loss='binary_crossentropy', metrics=['acc'])
print(STAMP)

early_stopping = EarlyStopping(monitor='val_loss', patience=3)  # 3轮无增长则停止
bst_model_path = STAMP + '.h5'
model_checkpoint = ModelCheckpoint(bst_model_path, save_best_only=True, save_weights_only=True)

lstm_189_125_0.25_0.20


In [24]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_3 (InputLayer)             (None, 30)            0                                            
____________________________________________________________________________________________________
input_4 (InputLayer)             (None, 30)            0                                            
____________________________________________________________________________________________________
embedding_2 (Embedding)          (None, 30, 100)       12041800    input_3[0][0]                    
                                                                   input_4[0][0]                    
____________________________________________________________________________________________________
lstm_2 (LSTM)                    (None, 189)           219240      embedding_2[0][0]       

In [30]:
len(train_data)

808580

In [35]:
hist = model.fit([train_data1, train_data2], train_label,
                 validation_data=([valid_data1, valid_data2], valid_label, valid_weight),
                 epochs=200, batch_size=2048, shuffle=True,
                 class_weight=class_weight, callbacks=[early_stopping, model_checkpoint])

model.load_weights(bst_model_path)
bst_val_score = min(hist.history['val_loss'])
wrs_val_score = max(hist.history['val_loss'])

Train on 727722 samples, validate on 80858 samples
Epoch 1/200


  2048/727722 [..............................] - ETA: 1045s - loss: 0.8957 - acc: 0.5005

  4096/727722 [..............................] - ETA: 574s - loss: 0.8948 - acc: 0.5146 

  6144/727722 [..............................] - ETA: 415s - loss: 0.8533 - acc: 0.5233

  8192/727722 [..............................] - ETA: 335s - loss: 0.8325 - acc: 0.5299

...

638976/727722 [=========================>....] - ETA: 12s - loss: 0.2694 - acc: 0.7949

641024/727722 [=========================>....] - ETA: 12s - loss: 0.2694 - acc: 0.7950

643072/727722 [=========================>....] - ETA: 12s - loss: 0.2694 - acc: 0.7950

645120/727722 [=========================>....] - ETA: 11s - loss: 0.2694 - acc: 0.7949

647168/727722 [=========================>....] - ETA: 11s - loss: 0.2694 - acc: 0.7949